### Library

In [4]:
from datetime import datetime, timedelta
import oandapy
from config import token, acc_id

### Oandaのインスタンス化

In [5]:
access_token = token
oanda = oandapy.API(environment="practice", access_token=access_token)

### 取引

In [6]:
# sample account_id
account_id = acc_id

# set the trade to expire after one day
trade_expire = datetime.utcnow() + timedelta(days=1)
trade_expire = trade_expire.isoformat("T") + "Z"

response = oanda.create_order(account_id,
    instrument='USD_JPY',
    units=100,
    side='buy',
    type='limit',
    price=110.30,
    expiry=trade_expire
)